In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from PIL import Image
import os

In [2]:
class ChestXrayDataset(Dataset):
    def __init__(self, image_folder, csv_file, transform=None):
        self.image_folder = image_folder
        self.data = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_folder, f'img_{self.data.iloc[idx, 0]}.png') 
        image = Image.open(img_name).convert('L')
        if self.transform:
            image = self.transform(image)
        label = int(self.data.iloc[idx, 1])
        return image, label

In [3]:
class DenseBlock(nn.Module):
    def __init__(self, in_channels, growth_rate, num_layers):
        super(DenseBlock, self).__init__()
        self.layers = nn.ModuleList()
        for i in range(num_layers):
            self.layers.append(self.conv_block(in_channels + i * growth_rate, growth_rate))

    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, bias=False)
        )

    def forward(self, x):
        features = [x]
        for layer in self.layers:
            x = layer(torch.cat(features, dim=1))
            features.append(x)
        return torch.cat(features, dim=1)


class Transition(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Transition, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        x = self.conv(x)
        x = self.pool(x)
        return x


class DenseNet121(nn.Module):
    def __init__(self, num_classes=3, growth_rate=32, block_config=(6, 12, 24, 16)):
        super(DenseNet121, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )

        num_features = 64
        for i, num_layers in enumerate(block_config):
            block = DenseBlock(num_features, growth_rate, num_layers)
            self.features.add_module('denseblock%d' % (i + 1), block)
            num_features = num_features + num_layers * growth_rate
            if i != len(block_config) - 1:
                trans = Transition(num_features, num_features // 2)
                self.features.add_module('transition%d' % (i + 1), trans)
                num_features = num_features // 2

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Linear(num_features, num_classes)

    def forward(self, x):
        features = self.features(x)
        out = self.avgpool(features)
        out = torch.flatten(out, 1)
        out = self.classifier(out)
        return out

In [14]:
from tqdm import tqdm

def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        tqdm_loader = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}')
        for images, labels in tqdm_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)
            tqdm_loader.set_postfix({'Loss': loss.item()})
        epoch_loss = running_loss / len(train_loader.dataset)
        torch.save(model.state_dict(), f'densenet121_2_{epoch + 14}.pth')
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_images_folder = "data/train_images/"
train_csv_file = "data/train_answers.csv"
batch_size = 32
num_epochs = 10

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_dataset = ChestXrayDataset(train_images_folder, train_csv_file, transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

model = DenseNet121(num_classes=3).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_model(model, train_loader, criterion, optimizer, num_epochs=num_epochs)

Epoch 1/10: 100%|██████████| 844/844 [1:13:33<00:00,  5.23s/it, Loss=0.594]


Epoch [1/10], Loss: 0.8320


Epoch 2/10: 100%|██████████| 844/844 [1:11:34<00:00,  5.09s/it, Loss=0.608]


Epoch [2/10], Loss: 0.5046


Epoch 3/10: 100%|██████████| 844/844 [1:11:34<00:00,  5.09s/it, Loss=0.674]


...

Epoch [8/10], Loss: 0.2766


Epoch 9/10: 100%|██████████| 844/844 [2:08:37<00:00,  9.14s/it, Loss=0.219]   


Epoch [9/10], Loss: 0.2675


Epoch 10/10: 100%|██████████| 844/844 [2:14:13<00:00,  9.54s/it, Loss=0.206]   

Epoch [10/10], Loss: 0.2455


In [6]:
torch.save(model.state_dict(), 'densenet121_1.pth')

In [11]:
train_model(model, train_loader, criterion, optimizer, num_epochs=4)

Epoch 1/4: 100%|██████████| 844/844 [2:14:52<00:00,  9.59s/it, Loss=0.293]   


Epoch [1/4], Loss: 0.2431


Epoch 2/4: 100%|██████████| 844/844 [2:15:00<00:00,  9.60s/it, Loss=0.106]   


Epoch [2/4], Loss: 0.2358


Epoch 3/4: 100%|██████████| 844/844 [2:14:34<00:00,  9.57s/it, Loss=0.162]   


Epoch [3/4], Loss: 0.2052


Epoch 4/4: 100%|██████████| 844/844 [1:51:00<00:00,  7.89s/it, Loss=0.112] 

Epoch [4/4], Loss: 0.1945


In [12]:
torch.save(model.state_dict(), 'densenet121_14.pth')

In [15]:
train_model(model, train_loader, criterion, optimizer, num_epochs=16)

Epoch 1/16: 100%|██████████| 844/844 [1:56:01<00:00,  8.25s/it, Loss=0.223]  


Epoch [1/16], Loss: 0.1859


Epoch 2/16: 100%|██████████| 844/844 [1:55:29<00:00,  8.21s/it, Loss=0.0289]


Epoch [2/16], Loss: 0.1797


Epoch 3/16: 100%|██████████| 844/844 [1:39:59<00:00,  7.11s/it, Loss=0.965]  


...

Epoch [8/16], Loss: 0.1213


Epoch 9/16: 100%|██████████| 844/844 [1:46:41<00:00,  7.58s/it, Loss=0.00253] 


Epoch [9/16], Loss: 0.1070


Epoch 10/16:  82%|████████▏ | 689/844 [1:28:37<19:56,  7.72s/it, Loss=0.125]   


KeyboardInterrupt: 

In [16]:
submission_df = pd.DataFrame(columns=['Id', 'target_feature'])
test_images_folder = "data/test_images/"
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

submission_data = []

for i, filename in enumerate(os.listdir(test_images_folder)):
    img_path = os.path.join(test_images_folder, filename)
    img = Image.open(img_path).convert('L')
    img = test_transform(img).unsqueeze(0).to(device)
    output = model(img)
    _, predicted = torch.max(output, 1)
    if predicted.item() is None or predicted.item() == '':
        target_feature = 0
    else:
        target_feature = predicted.item()
    submission_data.append({'id': i, 'target_feature': target_feature})

submission_df = pd.DataFrame(submission_data)
submission_df.to_csv('submission_file-densenet121_30.csv', index=False)